# Pyterrier - Example Experiment

# Preparation

In [1]:
#!pip install python-terrier
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-ootrg_ul/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-ootrg_ul/python-terrier
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.1.3-cp36-none-any.whl size=29595 sha256=8ce120bd9f33342a9335a6a50c62a7af372b56e009f689b49daceb1076cda604
  Stored in directory: /tmp/pip-ephem-wheel-cache-ge77gm06/wheels/cc/bb/69/836d846a92c787b35ca6478119c0033762ab2b95d866eeb288
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=42f53397a620947fedd9e3dbe6c7fff427cde4b293d81f00ca986bff2e98ccd3
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for pytrec-eval: filename=pytrec_eval-0.4-cp36-cp36m-linux_x86_64.whl size=273829 sha256=6693a7d374a09011262da29b1f

# Init 

You must run `pt.init()` before other pyterrier functions and classes

Arguments:
 - `version` - Terrier platform version e.g. `"5.2"`    
 - `mem` - megabytes allocated to Java e.g. `4096`      


In [2]:
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.2  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.2  jar not found, downloading to /root/.pyterrier...
Done


Again, we're using the Dataset interface to quickly access a test collection.

In [0]:
vaswani = pt.datasets.get_dataset("vaswani")

# Experiment

Тhe `experiment` function allows you to perform retrieval and evaluation in a declarative fashoin, allowing a simple function call with multiple retrieval systems to be executed using a single function call.

First create the BatchRetrieve objects with the properties you wish to use

In [4]:
TF_IDF = pt.BatchRetrieve(vaswani.get_index(), controls = {"wmodel": "TF_IDF"})
BM25 = pt.BatchRetrieve(vaswani.get_index(), controls = {"wmodel": "BM25"})
PL2 = pt.BatchRetrieve(vaswani.get_index(), controls = {"wmodel": "PL2"})

Call `pt.Experiment` with the topics, list of retrieval objects, list of metrics and qrels

Optional arguments:    
 - `perquery=False`, Show the results for each query instad of the mean    
 - `dataframe=True`, Return the result as a dataframe if True, or as a dictionary if False

In [9]:
eval = pt.pipelines.Experiment(
    vaswani.get_topics(),
    [TF_IDF,BM25,PL2],
    ['map','ndcg'],
    vaswani.get_qrels())
eval

,name,map,ndcg
0,BR(TF_IDF),0.290905,0.615367
1,BR(BM25),0.296517,0.621197
2,BR(PL2),0.276264,0.601225
